In [1]:
import pandas as pd
%matplotlib inline
import numpy as np
import geopy.distance


In [34]:
crime = 'crimes-in-boston/boston_safe_lat&long_filtered.csv'
crime = pd.read_csv(crime)
df = pd.DataFrame(crime)
df = df[['lat','long']]
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19582 entries, 0 to 19581
Data columns (total 2 columns):
lat     19582 non-null float64
long    19582 non-null float64
dtypes: float64(2)
memory usage: 306.1 KB


In [37]:
df_dt = df[(df.lat <= 42.366623) & (df.lat >=42.353177) | (df.long <= -71.051245) &(df.long >=-71.050987)]
df_dt.reset_index(drop=True,inplace=True)
df_dt.to_csv('crimes-in-boston/safe_downtown.csv')
df_dt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2728 entries, 0 to 2727
Data columns (total 2 columns):
lat     2728 non-null float64
long    2728 non-null float64
dtypes: float64(2)
memory usage: 42.8 KB


In [38]:
df_dt.head()

,lat,long
0,42.366584,-71.049143
1,42.366503,-71.049594
2,42.366354,-71.049503
3,42.366348,-71.049080
4,42.366282,-71.049087


In [39]:
df_dt['long'][2555]

-71.11527175100862

In [137]:
def getneighbors(startlocation):
    neighborDF = pd.DataFrame(columns=['ID', 'Distance'])
    coords_1 = (df_dt['lat'][startlocation], df_dt['long'][startlocation])
    for index, row in df_dt.iterrows():
        coords_2 = (df_dt['lat'][index], df_dt['long'][index])
        distance = round((geopy.distance.vincenty(coords_1, coords_2)).meters,2)
        neighborDF.loc[len(neighborDF)] = [index, distance]
    neighborDF = neighborDF.sort_values(by=['Distance'])
    neighborDF = neighborDF.reset_index(drop=True)

    return neighborDF[1:3]

In [139]:
graph={}
for i in range(1,10):
    graph[str(i)]={}
    s = getneighbors(i)
    for j in range(1,3):
        graph[str(i)][str(int(s.get_value(j, 'ID')))]=s.get_value(j, 'Distance')



/Users/shujiangtan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: Vincenty is deprecated and is going to be removed in geopy 2.0. Use `geopy.distance.geodesic` (or the default `geopy.distance.distance`) instead, which is more accurate and always converges.
  
/Users/shujiangtan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  


In [140]:
graph

{'1': {'2': 18.19, '0': 38.22},
 '2': {'1': 18.19, '3': 34.84},
 '3': {'4': 7.43, '5': 8.83},
 '4': {'5': 1.84, '6': 4.04},
 '5': {'4': 1.84, '6': 2.27},
 '6': {'5': 2.27, '7': 3.27},
 '7': {'8': 2.36, '6': 3.27},
 '8': {'7': 2.36, '9': 4.55},
 '9': {'8': 4.55, '7': 6.71}}

In [130]:
graph = {'1':{'2':3,'3':4, '4':7},
            '2':{'3':1,'6':5},
            '3':{'6':6,'4':2},
            '4':{'5':3, '7':6},
            '5':{'7':3, '8':4},
            '6':{'5':1, '8':8},
            '7':{'8':2},
            '8':{'7':2}
        }

In [135]:
def dijkstra(graph,start,goal):

    shortest_distance = {} 
    #dictionary to record the cost to reach to node. 
    #We will constantly update this dictionary as we move along the graph.
    track_predecessor = {} 
    #dictionary to keep track of path that led to that node.
    unseenNodes = graph 
    #to iterate through all nodes
    infinity = 10000 
    #infinity can be considered a very large number
    track_path = [] 
    #dictionary to record as we trace back our journey


# =============================================================================
# Initially we want to assign 0 as the cost to reach to source node and infinity as cost to all other nodes
# =============================================================================

    for node in unseenNodes:
        shortest_distance[node] = infinity

    shortest_distance[start] = 0

# =============================================================================
# The loop will keep running until we have entirely exhausted the graph, until we have seen all the nodes
# =============================================================================
# =============================================================================
# To iterate through the graph, we need to determine the min_distance_node every time.
# =============================================================================

    while unseenNodes:
        min_distance_node = None

        for node in unseenNodes:
            if min_distance_node is None:
                min_distance_node = node

            elif shortest_distance[node] < shortest_distance[min_distance_node]:
                min_distance_node = node

# =============================================================================
# From the minimum node, what are our possible paths
# =============================================================================

        path_options = graph[min_distance_node].items()


# =============================================================================
# We have to calculate the cost each time for each path we take 
#    and only update it if it is lower than the existing cost
# =============================================================================

        for child_node, weight in path_options:

            if weight + shortest_distance[min_distance_node] < shortest_distance[child_node]:

                shortest_distance[child_node] = weight + shortest_distance[min_distance_node]

                track_predecessor[child_node] = min_distance_node

# =============================================================================
# We want to pop out the nodes that we have just visited so that we dont iterate over them again.
# =============================================================================
        unseenNodes.pop(min_distance_node)



# =============================================================================
# Once we have reached the destination node, we want trace back our path and calculate the total accumulated cost.
# =============================================================================

    currentNode = goal

    while currentNode != start:

        try:
            track_path.insert(0,currentNode)
            currentNode = track_predecessor[currentNode]
        except KeyError:
            print('Path not reachable')
            break
    track_path.insert(0,start)


# =============================================================================
#  If the cost is infinity, the node had not been reached.
# =============================================================================
    if shortest_distance[goal] != infinity:
        print('Shortest distance is ' + str(shortest_distance[goal]))
        print('And the path is ' + str(track_path))


In [136]:
dijkstra(graph, '1', '8')

KeyError: '0'